In [ ]:
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)  
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', 800)

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
%cd /content/gdrive/My Drive/HW_Kaggle_NLP

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/My Drive/HW_Kaggle_NLP


In [ ]:
df = pd.read_csv('train_data.csv', sep=',')

In [ ]:
!pip install pymorphy2

In [ ]:
import re
import nltk
nltk.download('stopwords')
from pymorphy2 import MorphAnalyzer
from functools import lru_cache
from nltk.corpus import stopwords

m = MorphAnalyzer()
regex = re.compile("[А-Яа-яA-z]+")

def words_only(text, regex=regex):
    try:
        return regex.findall(text.lower())
    except:
        return []

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
@lru_cache(maxsize=128)
def lemmatize_word(token, pymorphy=m):
    return pymorphy.parse(token)[0].normal_form

def lemmatize_text(text):
    return [lemmatize_word(w) for w in text]


mystopwords = stopwords.words('russian') 
def remove_stopwords(lemmas, stopwords = mystopwords):
    return [w for w in lemmas if not w in stopwords and len(w) > 3]

def clean_text(text):
    tokens = words_only(text)
    lemmas = lemmatize_text(tokens)
    
    return ' '.join(remove_stopwords(lemmas))

In [ ]:
from multiprocessing import Pool
from tqdm import tqdm

with Pool(4) as p:
    lemmas = list(tqdm(p.imap(clean_text, df['comment']), total=len(df)))
    
df['lemmas'] = lemmas
df.sample(5)

100%|██████████| 10809/10809 [01:08<00:00, 158.30it/s]


,comment,toxic,lemmas
8967,"Ну, средняя зарплата в Челябинской области на начало 2019 года 33 т.р. Продолжительность жизни 70 лет. Эти и другие мифы статистики смотрите в следующей серии. Даже по данной таблице видно что цена выросла.\n",0.0,средний зарплата челябинский область начало продолжительность жизнь статистика смотреть следующий серия дать таблица видно цена вырасти
7289,"Среагируют, но смогут ли что-то сделать - это вопрос. Сбить падающую из космоса хрень пока получилось только у американцев Ещё китайцы сбивали свои спутники на орбите. Думаю, больше ни у кого таких возможностей нет.\n",0.0,среагировать смочь сделать вопрос сбить падать космос хрень пока получиться американец китаец сбивать свой спутник орбита думать большой возможность
2749,Политолог-единоросс из маргинальной конфы обозревает ослепительную победу Стаса671Какпросто над девственным Павуком:\n,0.0,политолог единоросс маргинальный конф обозревать ослепительный победа стас какпросто девственный павук
3036,Не такая жизнь у сисадмина.. .На самом деле не опытные пользователи постоянно консультироваться звонят. Не до танков.\n,0.0,жизнь сисадмин дело опытный пользователь постоянно консультироваться звонить танк
7516,спасибо большое) нужно послушать песню\n,0.0,спасибо большой нужно послушать песня


In [ ]:
df['toxic'] = df.toxic.astype(int)
df['comment'] = df.lemmas
train = pd.DataFrame(df[['comment', 'toxic']])
x_train = train['comment']
y_train = train['toxic']
x_train

0                                                                                                                                                                                                                                                                                               преступление наказание
1                                                                                                                                                                                                                                                                         именно неработающий весы показывать работать
2                                                                                                                                                                                                                                                                                              япония панелька ебанько
3                                                                  

In [ ]:
train

,comment,toxic
0,преступление наказание,0
1,именно неработающий весы показывать работать,0
2,япония панелька ебанько,0
3,выявлять трещина помощь белый краска магнитная краска прислонять большой магнит трещина проявляться знать метод называться труба проверять,0
4,дочитать поезд норильск далёкий стать,0
...,...,...
10804,мама группа выпуск просяк случаться разный ребёнок ребёнок разовый акция время время третий частенько родитель прекрасно курс особенность каждый конкретный ребёнок,0
10805,сука тупой дегенарта видео съести свой старый куколд жухлый сморчок друг друг теребить,1
10806,племя украинец особенно западный детство прививаться мысль самый умный ловко наебал значит молодец понятие подлость честь отсутствовать нацело поэтому маленький дружок весь твой натужный изворотливость работать пообщаться пять минута любой россия пойма сорт иметь дело услышать мягкий акцент твой речь,1
10807,пост жадность человек оплатить предоставить халява человек предвкушать,0


In [ ]:
xt1 = pd.read_csv('test_data.csv', sep=',').drop('comment_id', axis=1)
xt1

,comment
0,"В данном случае не пионервожатая, а техничка, полагаю хвастать о сексе с техничкой стоит исключительно с озвучиванием возраста, а лучше не хвастать вовсе, никогда, ни о чем...\n"
1,"В данном посте рассматривается обычный ленточный фундамент без специальных мероприятий. И да, применение гидрофобных материалов используется, но как мероприятие снижающее силу морозного пучения, а не исключающее его. Его надо как-то посчитать. И этот материал должен обеспечивать заданные характеристики на весь срок службы сооружения. Некоторые производители подтверждают испытаниями, что их экструдированный пенополистирол проживет 50 лет. Но он достаточно жесткий, чтобы передать и нормальные и касательные силы. Поэтому я бы лучше утеплил отмостку, как это рекомендует ПНИИИС (см. рис).\n"
2,"Как я понимаю, у поверхностей отражаемость поглощаемость разная. Типо нельзя прям до градусов температуру разных материалов сравнивать.\n"
3,Австралийские пауки съедят все живое на планете))
4,"У нас раньше Амур пиво целые бутылки принимал по 1.50 свои, 1р. чужие и 50коп лом. Так что у нас всё это было в 90х\n"
...,...
3598,"Я всё жду, когда эта херня закончится\n"
3599,"Откуда такой акцент на Москве? У нас между прочим многонациональное государство по конституции. И эту свою ахинею они исполняли не в открытом доступе, где нибудь на красной площади, а на арендованной площадке. Думаю, все кто хотел пойти, мог почитать предварительно, что это за хрень.Мы требуем, чтобы уважали нас, при этом готовы плюнуть на других? Это про нас!\n"
3600,"Кого угодно, но не этих двух отбросов: свинью и недоразвитого дауна.\n"
3601,А как же радиоуправляемые машинки в клубах?\n


In [ ]:
with Pool(4) as p:
    lemmas = list(tqdm(p.imap(clean_text, xt1['comment']), total=len(xt1)))
xt1['lemmas'] = lemmas    

100%|██████████| 3603/3603 [00:17<00:00, 201.54it/s]


In [ ]:
xt1['comment'] = xt1.lemmas
x_test = xt1['comment'] 
x_test

0                                                                                                                                                                                                                                                                                                                                                            данный случай пионервожатый техничка полагать хвастать секс техничка стоить исключительно озвучивание возраст хороший хвастать вовсе
1       данный пост рассматриваться обычный ленточный фундамент специальный мероприятие применение гидрофобный материал использоваться мероприятие снижать сила морозный пучение исключать посчитать материал должный обеспечивать задать характеристика весь срок служба сооружение некоторый производитель подтверждать испытание экструдировать пенополистирол прожить достаточно жёсткий передать нормальный касательный сила поэтому хороший утеплить отмостка рекомендовать пнииис смотреть
2                                   

In [ ]:
from sklearn.linear_model import LogisticRegression 
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
from nltk import ngrams

In [ ]:
vec = CountVectorizer(ngram_range=(1, 1)) # строим BoW для слов
bow = vec.fit_transform(x_train) 

In [ ]:
list(vec.vocabulary_.items())[:10]

[('преступление', 18103),
 ('наказание', 12158),
 ('именно', 8041),
 ('неработающий', 13088),
 ('весы', 2977),
 ('показывать', 16994),
 ('работать', 19422),
 ('япония', 27020),
 ('панелька', 15266),
 ('ебанько', 6372)]

In [ ]:
bow[0]

<1x27057 sparse matrix of type '<class 'numpy.int64'>'
	with 2 stored elements in Compressed Sparse Row format>

In [ ]:
clf = LogisticRegression(random_state=42, max_iter=500)
clf.fit(bow, y_train)

LogisticRegression(max_iter=500, random_state=42)

In [ ]:
pred = clf.predict(vec.transform(x_test))
pred

array([0, 0, 0, ..., 1, 0, 0])

In [ ]:
subm_3 = [[i, pred[i]] for i in range(len(x_test))]
subm_3 = pd.DataFrame(subm_3, columns=['comment_id', 'toxic'])

subm_3.to_csv('submission_3.csv', index=False)
pd.read_csv('submission_3.csv', sep=',').head(50)

,comment_id,toxic
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0
5,5,0
6,6,0
7,7,0
8,8,1
9,9,0


 + fasttext -> 6

In [ ]:
!pip install fasttext

In [ ]:
import fasttext

In [ ]:
with open('train_ft.txt', 'w') as f:
    for pair in list(zip(train['comment'], train['toxic'])):
        text, label = pair
        f.write(f'__label__{label} {text.lower()}\n')

In [ ]:
classifier = fasttext.train_supervised('train_ft.txt')


In [ ]:
pred1 = classifier.predict(list(x_test))[0]
pred1 = [int(label[0][-1]) for label in pred1]

In [ ]:
subm_4 = [[i, pred1[i]] for i in range(len(x_test))]
subm_4 = pd.DataFrame(subm_4, columns=['comment_id', 'toxic'])

subm_4.to_csv('submission_4.csv', index=False)

In [ ]:
pd.read_csv('submission_4.csv', sep=',').head(50)

,comment_id,toxic
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0
5,5,0
6,6,0
7,7,0
8,8,1
9,9,0
